In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
#set up file paths
file_path1 = os.path.join("raw_data", "schools_complete.csv")
file_path2 = os.path.join("raw_data", "students_complete.csv")

In [3]:
#define files
file1 = pd.read_csv(file_path1)
file2 = pd.read_csv(file_path2)

In [50]:
#create dataframes
df_school = pd.DataFrame(file1)
df_stud = pd.DataFrame(file2)

df_school


,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [5]:
#Change column headers in both dfs to allow for merge
df_school_sync = df_school.rename(columns={"name":"School Name"})
df_stud_sync = df_stud.rename(columns={"school":"School Name"})

df_stud_sync.head()

,Student ID,name,gender,grade,School Name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [6]:
#right join tables by School Name, including all student table rows
merged_data = pd.merge(df_school_sync, df_stud_sync, on="School Name", how="right")

#check out the merged table
merged_data.head()

,School ID,School Name,type,size,budget,Student ID,name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [7]:
#create metrics
#count total # of schools by unique names
total_schools = merged_data["School Name"].nunique()

#count total students by length of names column
total_students = len(merged_data["name"])
total_students

#total budget by sum of 
total_budget = df_school_sync["budget"].sum()
total_budget

#calc average math & reading score
avg_math = merged_data["math_score"].mean()
#calc average math score
avg_reading = merged_data["reading_score"].mean()
avg_reading

#define passing level
pass_score = 70

#calculate number of students passing math and reading
reading_pass_stud = merged_data["reading_score"].loc[merged_data["reading_score"] > int(pass_score)].count()
math_pass_stud = merged_data["math_score"].loc[merged_data["math_score"] > int(pass_score)].count()

#calculate percentages
pct_passing_math = math_pass_stud / total_students
pct_passing_reading = reading_pass_stud/ total_students
avg_pass_rate = (pct_passing_reading + pct_passing_math) / 2
avg_pass_rate

0.77681899412815936

In [8]:
district_summary = pd.DataFrame({"Total Schools":total_schools, "Total Students":total_students, "Total Budget":total_budget,
                                "Average Math Score":avg_math, "Average Reading Score":avg_reading, "% Passing Math":pct_passing_math, 
                                 "% Passing Reading":pct_passing_reading, "Overall Pass Rate":avg_pass_rate}, index=[0])
district_summary.head()

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Pass Rate,Total Budget,Total Schools,Total Students
0,0.723921,0.829717,78.985371,81.87784,0.776819,24649428,15,39170


In [52]:
#UNNECESSARY... THIS IS THE HARD WAY (easy way is later)

#create new data frame to store school-level data
school_avg_data = merged_data.groupby(["School Name"]).mean()
school_avg_data_sorted = school_avg_data.sort_values("School ID")
school_avg_data_sorted.reset_index(inplace=True)
school_avg_data_sorted.head()



,School Name,School ID,size,budget,Student ID,reading_score,math_score
0,Huang High School,0.0,2917.0,1910635.0,1458.0,81.182722,76.629414
1,Figueroa High School,1.0,2949.0,1884411.0,4391.0,81.158020,76.711767
2,Shelton High School,2.0,1761.0,1056600.0,6746.0,83.725724,83.359455
3,Hernandez High School,3.0,4635.0,3022020.0,9944.0,80.934412,77.289752
4,Griffin High School,4.0,1468.0,917500.0,12995.5,83.816757,83.351499


In [56]:
#I MIGHT NEED TO RE-DO THIS BY PULLING I FROM MERGED DATA... using original tables isn't really working... maybe sort?

#School-Level Metrics
school_names = df_school_sync["School Name"]

#designate school types
school_types = df_school_sync["type"]

#number of students by school
school_size = df_school_sync["size"]

#total budget by school
school_budget = df_school_sync["budget"]

#per student budget
per_student_budget = school_budget / school_size

#average math score
avg_school_math = school_avg_data["math_score"]

#average reading score
avg_school_reading = school_avg_data["reading_score"]

#% group the data frame to show the number that passed math for each school... grouping by name caused problems so went with ID
school_math_pass_only= merged_data.loc[(merged_data["math_score"] > 70)].groupby("School ID").count()

#create a list that is only the math scores per school (other columns gone)
school_math_pass_tot = school_math_pass_only["math_score"]

#calculate percentage passing math for each school
school_math_pass_pct = school_math_pass_tot / school_size

#% passing reading
#% group the data frame to show the number that passed reading for each school... grouping by name caused problems so went with ID
school_reading_pass_only= merged_data.loc[(merged_data["reading_score"] > 70)].groupby("School ID").count()

#create a list that is only the reading scores per school (other columns gone)
school_reading_pass_tot = school_reading_pass_only["reading_score"]

#calculate percentage passing reading for each school
school_reading_pass_pct = school_reading_pass_tot / school_size
school_reading_pass_pct

#%overall passing rate
school_overall_pass_rate = (school_math_pass_pct + school_reading_pass_pct) / 2

school_types_list = school_types.tolist()
print(school_types_list)

#Create a new data frame that stores all of it
school_summary_data = pd.DataFrame({
    "": school_names.tolist(),
    "School Type": school_types.tolist(),
    "Total Students": school_size.tolist(),
    "Total School Budget":school_budget.tolist(),
    "Per Student Budget":per_student_budget.tolist(),
    "Average Math Score":avg_school_math.tolist(),
    "Average Reading Score":avg_school_reading.tolist(),
    "% Passing Math":school_math_pass_pct.tolist(),
    "% Passing Reading":school_reading_pass_pct.tolist(),
    "% Overall Passing Rate":school_overall_pass_rate.tolist()
    }).sort_values("")

school_summary_data.set_index("",inplace=True)
school_summary_data.head()
#cols = school_summary_data.columns.values


['District', 'District', 'Charter', 'District', 'Charter', 'Charter', 'Charter', 'District', 'Charter', 'Charter', 'Charter', 'District', 'District', 'District', 'Charter']


,% Overall Passing Rate,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Per Student Budget,School Type,Total School Budget,Total Students
,,,,,,,,,
Bailey High School,0.719654,0.646302,0.793006,76.629414,81.182722,628.0,District,3124928,4976
Cabrera High School,0.917115,0.895587,0.938644,83.803279,83.814988,582.0,Charter,1081356,1858
Figueroa High School,0.710919,0.637504,0.784334,83.061895,83.975780,639.0,District,1884411,2949
Ford High School,0.716320,0.657539,0.775100,83.274201,83.989488,644.0,District,1763916,2739
Griffin High School,0.915531,0.897139,0.933924,83.351499,83.816757,625.0,Charter,917500,1468
